# Reducing the size of the dataset 

## 1 Separation of queries: 
We should find a way to return graphs of posts, users and subreddits, one subreddit at a time. This is computationally easier to handle for our pc's. The queries will be simpler, and it will be more manual work. However once we have all these separate files we can merge them together into a big connected graph. It is not clear to me yet which approach is easiest.
1.  Exporting all the data in a csv and merging the csv's together and importing the data into gephi OR
2.  Exporting the graphml files and merging them


## 2 Reducing Complexity of the database

### 2.1 User short list
We generated two approaches to arrive to a shortlist of users to analyse

#### 2.1.1 Subreddit driven approach: 
For each of the three main corona subreddits we need to return the top 100 posts with the highest karma. For these 300 posts we extract the users that made them. These are intensive reddit users that have a high Impact within these three subreddits.



**NEO4J QUERY**

**coronavirus**
```
MATCH (u:User)--(p:Post)--(s:Subreddit {display_name:'coronavirus'})
RETURN p.title, p.score,  s.display_name,u.username
ORDER BY p.score DESC
LIMIT 100
```

**covid19**
```
MATCH (u:User)--(p:Post)--(s:Subreddit {display_name:'coronavirus'})
RETURN p.title, p.score,  s.display_name,u.username
ORDER BY p.score DESC
LIMIT 100
```

**china_flu**
```
MATCH (u:User)--(p:Post)--(s:Subreddit {display_name:'coronavirus'})
RETURN p.title, p.score,  s.display_name,u.username
ORDER BY p.score DESC
LIMIT 100
```

**Function defintions**

In [2]:
def dates(df):
    "Takes UTC dates and returns columns for the months, weeks "
    df['month'] = pd.to_datetime(arg=df.loc[:, 'date']).dt.month_name()
    df['month_n'] = pd.to_datetime(arg=df.loc[:, 'date']).dt.month
    df['day'] = pd.to_datetime(arg=df.loc[:, 'date']).dt.day_name()
    df['day_y'] = pd.to_datetime(arg=df.loc[:, 'date']).dt.dayofyear
    df['day_w'] = pd.to_datetime(arg=df.loc[:, 'date']).dt.dayofweek
    df['week_y'] = pd.to_datetime(arg=df.loc[:, 'date']).dt.weekofyear
    
    return df

**Extract list of users from the three main corona subreddits**

In [66]:
DATA_DIR = 'C:\\Users\\delah\\Documents\\Programming\\workspace\\AAForB_Assignment4\\data'
ANALYSIS_DIR = '\\users'
INPUT_DIR = '\\input'
OUTPUT_DIR = '\\output'
VIS_DIR = 'C:\\Users\\delah\\Documents\\Programming\\workspace\\AAForB_Assignment4\\outputs\\exploratory_analysis\\users\\'

os.chdir(DATA_DIR + ANALYSIS_DIR + INPUT_DIR)

files = ['china_flu','coronavirus','covid19']

users = pd.DataFrame()
for file in files:
    temp = pd.read_csv(file)
    users = users.append(temp)
    
users = users.rename(columns={'p.title':'title','s.display_name':'subreddit','p.score':'score','u.username':'username'})


usernames = users.username.values
#print(usernames)  # print this to get the full list in Neo4J compatible format
#users = users.iloc[0:50,:] # We reduce the list of users to limit the size of the final graph down the line
subreddit_driven = users


print("Users list shape:", users.shape)

Users shape: (300, 4)
               score
count     300.000000
mean    22307.620000
std     29433.558186
min       664.000000
25%      1690.750000
50%      3105.000000
75%     49307.500000
max    110424.000000 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 300 entries, 0 to 99
Data columns (total 4 columns):
title        300 non-null object
score        300 non-null int64
subreddit    300 non-null object
username     300 non-null object
dtypes: int64(1), object(3)
memory usage: 11.7+ KB
None


**Save data to directory**

In [4]:
os.chdir(DATA_DIR + ANALYSIS_DIR + OUTPUT_DIR)
users.to_csv('subreddit_driven.csv')

#### 2.2.2 user driven approach: 
Return the list of users that have posted in all three subreddits, Of this list, take those with the highest karma.
There will be some overlap in users between the subreddit driven approach and the user driven approach which is good. The former emphasize highly contributing members within each community, while the latter emphasizes on members that connect communities (with or without reposts).

**NEO4J QUERY**
```
match (sr:Subreddit)--(p:Post)--(u:User)
where sr.display_name in ['coronavirus', 'covid19', 'china_flu']
and u.username <> 'AutoModerator'
with distinct u as myUsers 

match (myUsers:User)--(pAny:Post)--(srAny:Subreddit)
where srAny.display_name in ['europe', 'lifeprotips', 'science', 'videos', 'technology', 'iama', 'todayilearned', 'coronavirus', 'askreddit', 'explainlikeimfive', 'news', 'china_flu', 'covid19', 'nottheonion', 'politics', 'upliftingnews', 'askscience', 'worldnews', 'dataisbeautiful']
with myUsers as my_N_Users, count(distinct pAny) as cntDiffPosts
order by cntDiffPosts desc, my_N_Users.username limit 300

return my_N_Users.username

```

In [68]:
os.chdir(DATA_DIR + ANALYSIS_DIR + OUTPUT_DIR)

file = 'user_driven.csv'

users = pd.read_csv(file)
    
users = users.rename(columns={'my_N_Users':'username','cntDiffPosts':'post_count'})



usernames = users.username.values
#print(usernames)  # print this to get the full list in Neo4J compatible format

#users = users.iloc[0:60,:]
user_driven = users

print("Users shape:", users.shape)
print(users.describe(), '\n')
print(users.info())

Users shape: (300, 2)
       post_count
count   300.00000
mean    300.98000
std     354.25997
min     138.00000
25%     171.75000
50%     215.00000
75%     299.50000
max    4900.00000 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 2 columns):
username      300 non-null object
post_count    300 non-null int64
dtypes: int64(1), object(1)
memory usage: 4.8+ KB
None


In [69]:
print('Length of the user driven approach list:' + str(len(user_driven.loc[:,['username']])))
print('Length of the subreddit driven approach list' + str(len(subreddit_driven.loc[:,['username']])))
users_master_list = user_driven.loc[:,['username']].append(subreddit_driven.loc[:,['username']])
print('Length of both lists:' + str(len(users_master_list)))
print('Length of both lists when removing duplicates' + str(len(set(users_master_list.values.flatten()))))
os.chdir(DATA_DIR + ANALYSIS_DIR + OUTPUT_DIR)
users_master_list.to_csv('users_master_list.csv')

Length of the user driven approach list:300
Length of the subreddit driven approach list300
Length of both lists:600
Length of both lists when removing duplicates512


In [71]:
usernames = list(set(users_master_list.values.flatten())) # print this to get the full list in Neo4J compatible format
print(len(usernames))
#usernames

512


### 2.2 subreddits: 
There are too many subreddits to query. Even with our user's shorts, if we query all the posts they make and their respective subreddits we will have a data explosion. Only limiting ourselves to the three main subreddits seems too
limiting too. We limit our queries to the subreddits from a new shortlist I made based on the "Biggest subreddits" list I posted above. The new list is shorted and contains the 7 subreddits that have the most posts related to corona.

In [8]:
subreddits

NameError: name 'subreddits' is not defined

**Don't forget to add datascience and machinelearning**

### 2.3 posts:
While many subreddits like /r/news have fewer than 5000 posts made for the entire period of analysis, others like /r/coronavirus have 5000 posts just for the april period. Again the goal is here to filter on posts that cover the coronacrisis.
However we want to limit that pool even further to reduce the total complexity of the file.
       @François Chandelle Let me know what you think of this.
#### 2.3.1 for the three mainsubreddits:
we export all the posts made by our shortlist of users


**NEO4J QUERIES:**

**coronvirus**
```
MATCH (u:User)-[e2]-(p:Post)-[e1]-(s:Subreddit{display_name:'coronavirus'})
WHERE u.username in //Add master list of users here
RETURN u.id, u.username, u.link_karma, u.comment_karma , s.id, s.display_name,p.id, p.created_utc_str, p.score, p.upvote_ratio ,p.title
ORDER BY p.score DESC
LIMIT 10000
```


**covid19**
```
MATCH (u:User)-[e2]-(p:Post)-[e1]-(s:Subreddit{display_name:'covid19'})
WHERE u.username in //Add master list of users here
RETURN u.id, u.username, u.link_karma, u.comment_karma , s.id, s.display_name,p.id, p.created_utc_str, p.score, p.upvote_ratio ,p.title
ORDER BY p.score DESC
LIMIT 10000
```

**china_flu**
```
MATCH (u:User)-[e2]-(p:Post)-[e1]-(s:Subreddit{display_name:'china_flu'})
WHERE u.username in //Add master list of users here
RETURN u.id, u.username, u.link_karma, u.comment_karma , s.id, s.display_name,p.id, p.created_utc_str, p.score, p.upvote_ratio ,p.title
ORDER BY p.score DESC
LIMIT 10000
```


In [77]:
# Point to directory containing data
DATA_DIR = 'C:\\Users\\delah\\Documents\\Programming\\workspace\\AAForB_Assignment4\\data'
ANALYSIS_DIR = '\\coronavirustrends'
INPUT_DIR = '\\input'
OUTPUT_DIR = '\\output'
VIS_DIR = 'C:\\Users\\delah\\Documents\\Programming\\workspace\\AAForB_Assignment4\\outputs\\exploratory_analysis\\coronatrends\\'


os.chdir(DATA_DIR + ANALYSIS_DIR + INPUT_DIR)

# Importing data and renaming columns
df = pd.DataFrame()
for i in range(1,20):
    temp = pd.read_csv(str(i))
    df = df.append(temp)
    
df = df.rename(columns={'p.title':'title','s.display_name':'subreddit','p.created_utc_str':'date','p.score':'score'})

# Extract subreddits
subreddits = list(set(df.loc[:,['subreddit']].values.flatten()))
print("Number of subreddits extracted:", len(list(subreddits)))
print(subreddits)

# Get months, weeks and days
df = dates(df)

print(df.shape)
df.head()
              

Number of subreddits extracted: 19
['iama', 'explainlikeimfive', 'technology', 'covid19', 'askscience', 'askreddit', 'china_flu', 'politics', 'dataisbeautiful', 'europe', 'videos', 'worldnews', 'coronavirus', 'upliftingnews', 'nottheonion', 'todayilearned', 'news', 'science', 'lifeprotips']
(32303, 10)


,title,subreddit,date,score,month,month_n,day,day_y,day_w,week_y
0,Redditors bored because coronavirus cancelled ...,askreddit,2020-03-13 15:17:31,3,March,3,Friday,73,4,11
1,Why would a second coronavirus wave breakout i...,askreddit,2020-04-13 05:58:50,1,April,4,Monday,104,0,16
2,What good things are happening right now but i...,askreddit,2020-04-15 00:10:04,1,April,4,Wednesday,106,2,16
3,"So now, what is the most memorable fake news y...",askreddit,2020-03-28 21:16:05,1,March,3,Saturday,88,5,13
4,people who had to use the health system during...,askreddit,2020-03-23 23:45:51,1,March,3,Monday,83,0,13


In [9]:
import pandas as pd
import os

# Setting the directories
DATA_DIR = 'C:\\Users\\delah\\Documents\\Programming\\workspace\\AAForB_Assignment4\\data'
ANALYSIS_DIR = '\\posts'
INPUT_DIR = '\\input'
OUTPUT_DIR = '\\output'
VIS_DIR = 'C:\\Users\\delah\\Documents\\Programming\\workspace\\AAForB_Assignment4\\outputs\\exploratory_analysis\\posts\\'
os.chdir(DATA_DIR + ANALYSIS_DIR + INPUT_DIR)

# Loading the data

files = ['china_flu_complete', 'coronavirus_complete', 'covid19_complete','technology',  'nottheonion',  'videos',  'politics',  'dataisbeautiful',  'askscience',  'news',  'askreddit',  'worldnews',  'todayilearned',  'upliftingnews',  'science',  'explainlikeimfive',  'europe']
#files = ['china_flu', 'coronavirus', 'covid19','technology',  'nottheonion',  'videos',  'politics',  'dataisbeautiful',  'askscience',  'news',  'askreddit',  'worldnews',  'todayilearned',  'upliftingnews',  'science',  'explainlikeimfive',  'europe'] # francois version with all posts from the main three
posts = pd.DataFrame()
for file in files:
    temp = pd.read_csv(file)
    posts = posts.append(temp)

#Cleaning posts dataframe
posts = posts.rename(columns={'p.title':'title','s.display_name':'subreddit','p.score':'score','u.username':'username','u.link_karma':'link_karma','u.comment_karma':'comment_karma', 'p.created_utc_str':'date','p.upvote_ratio':'upvote_ratio'})
posts = dates(posts)

#Groupby's 
# Users
total_posts_per_user = posts.loc[:,['username','title']].groupby('username').count().sort_values(by='title',  ascending=False)
total_posts_per_user = total_posts_per_user.rename(columns={'title':'total_posts_per_user'}).reset_index() 
total_scores_per_user = posts.loc[:,['username','score']].groupby('username').sum().sort_values(by='score', ascending=False).reset_index() 
total_scores_per_user = total_scores_per_user.rename(columns={'score':'total_score_per_user'})

#Subreddit 
total_posts_per_subreddit = posts.loc[:,['subreddit','title']].groupby('subreddit').count().sort_values(by='title',  ascending=False)
total_posts_per_subreddit = total_posts_per_subreddit.rename(columns={'title':'total_posts_per_subreddit'}).reset_index() 
total_scores_per_subreddit = posts.loc[:,['subreddit','score']].groupby('subreddit').sum().sort_values(by='score', ascending=False).reset_index() 
total_scores_per_subreddit = total_scores_per_subreddit.rename(columns={'score':'total_scores_per_subreddit'})

posts = pd.merge(left=posts,right=total_posts_per_user,how='left',left_on='username',right_on='username')
posts = pd.merge(left=posts,right=total_scores_per_user,how='left',left_on='username',right_on='username')
posts = pd.merge(left=posts,right=total_posts_per_subreddit,how='left',left_on='subreddit',right_on='subreddit')
posts = pd.merge(left=posts,right=total_scores_per_subreddit,how='left',left_on='subreddit',right_on='subreddit')
posts['average_karma_per_post'] = posts.loc[:,'total_score_per_user'] / posts.loc[:,'total_posts_per_user']
posts.columns


#Saving Dataframe
os.chdir(DATA_DIR + ANALYSIS_DIR + OUTPUT_DIR)
#posts = posts.loc[posts.username.isin(values=usernames)] # Special filter on shorter user list
posts['Time Interval'] = posts.loc[:,'day_y']
posts.to_csv('complete_graph.csv')
print(posts.shape)
posts.columns


(37386, 23)


Index(['u.id', 'username', 'link_karma', 'comment_karma', 's.id', 'subreddit',
       'p.id', 'date', 'score', 'upvote_ratio', 'title', 'month', 'month_n',
       'day', 'day_y', 'day_w', 'week_y', 'total_posts_per_user',
       'total_score_per_user', 'total_posts_per_subreddit',
       'total_scores_per_subreddit', 'average_karma_per_post',
       'Time Interval'],
      dtype='object')

**Final DataFrame containing topic**

In [11]:
# Setting the directories
DATA_DIR = 'C:\\Users\\delah\\Documents\\Programming\\workspace\\AAForB_Assignment4\\data'
ANALYSIS_DIR = '\\posts'
INPUT_DIR = '\\input\\'
OUTPUT_DIR = '\\output'
import os
os.chdir(DATA_DIR + ANALYSIS_DIR + INPUT_DIR)
final_graph = pd.read_csv('reddit_posts_with_topic_and_sentiment_4.csv',encoding='iso-8859-1')
final_graph.drop(labels=['Unnamed: 0'], axis=1, inplace=True)
final_graph.head()

,u.id,username,link_karma,comment_karma,s.id,subreddit,p.id,date,score,upvote_ratio,...,day_w,week_y,total_posts_per_user,total_score_per_user,total_posts_per_subreddit,total_scores_per_subreddit,average_karma_per_post,avg_sentiment,topic,topic_score
0,2o0ngbg8,goddessofthebitches,8376,1346,2dar36,china_flu,fhxap2,2020-03-13T10:05:33Z,16818,0.85,...,4,11,1,16818,9515,804324,16818.000000,0.000000,American Politics and News,57.0
1,tblkklw,DeWallenVanWimKok,21356,6296,2dar36,china_flu,f0p5nc,2020-02-08T08:59:21Z,11927,0.95,...,5,6,157,27710,9515,804324,176.496815,0.150000,Medical Research and Vaccine,47.0
2,14mosq,staplehill,21357,42254,2dar36,china_flu,fbt49e,2020-03-01T12:01:27Z,8922,0.98,...,6,9,14,12187,9515,804324,870.500000,0.103280,Medical Research and Vaccine,33.0
3,14bafgnj,IcyPresence96,4160,535,2dar36,china_flu,feupgd,2020-03-07T12:49:04Z,7043,0.91,...,5,10,4,7074,9515,804324,1768.500000,0.232119,American Politics and News,64.0
4,wo4fw,madman320,48415,22054,2dar36,china_flu,f30lyn,2020-02-12T23:49:03Z,6811,0.96,...,2,7,168,33525,9515,804324,199.553571,0.374166,Statistics Reporting,73.0


### 2.3.2 for the other subredditss: 
We only export the posts made by our shortlist of users containing topics extracted from the LDA made by victor

In [12]:
#TODO: 

# Gephi converter
To create the final graph we need to convert our data to csv. We will need two types of files, csv containing the node infromation and csv's containing the relationship information. Below we create a csv for each


## Nodes table

In [13]:
# Creating the nodes dataframe
nodes = final_graph.copy()

#Processing nodes
# Post nodes
nodes_post = nodes.loc[:,['p.id','title','date','score','upvote_ratio','month','month_n','day','Time Interval','day_w','week_y','avg_sentiment', 'topic', 'topic_score']]
nodes_post = nodes_post.rename(columns={'p.id':'Id','title':'Label'})
nodes_post.loc[:,'Label'] = ' '
nodes_post['node_type'] = 'post'
nodes_post = nodes_post.set_index(keys='Id')


# User nodes
nodes_user = nodes.loc[:,['u.id','username','link_karma','comment_karma','total_posts_per_user','Time Interval','total_score_per_user','average_karma_per_post']]
nodes_user = nodes_user.rename(columns={'u.id':'Id','username':'Label'})
nodes_user['node_type'] = 'user'
nodes_user = nodes_user.set_index(keys='Id')
# Subreddit nodes
nodes_subreddit = nodes.loc[:,['s.id','subreddit','total_posts_per_subreddit','Time Interval','total_scores_per_subreddit']]
nodes_subreddit = nodes_subreddit.rename(columns={'s.id':'Id','subreddit':'Label'})
nodes_subreddit['node_type'] = 'subreddit'
nodes_subreddit = nodes_subreddit.set_index(keys='Id')

C:\Users\delah\Anaconda3\lib\site-packages\pandas\core\indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


## Relationships Table

In [14]:
# Creating the relationships dataframe
relationships = final_graph.loc[:,['u.id','s.id','p.id']]

# Processing relationships
# User to post relationships
rel_user_to_post = nodes.loc[:,['u.id','p.id']]
rel_user_to_post['Submitted'] = 'Submitted'
rel_user_to_post = rel_user_to_post.rename(columns={'u.id':'Source','p.id':'Target'})

# Post to Subreddit relationships
rel_post_to_subreddit = nodes.loc[:,['p.id','s.id']]
rel_post_to_subreddit['Submitted'] = 'Submitted'
rel_post_to_subreddit = rel_post_to_subreddit.rename(columns={'p.id':'Source','s.id':'Target'})


print(nodes_post.shape)
print(nodes_user.shape)
print(nodes_subreddit.shape)
print(rel_user_to_post.shape)
print(rel_post_to_subreddit.shape)

(30337, 14)
(30337, 8)
(30337, 5)
(30337, 3)
(30337, 3)


### Saving data

In [15]:
DATA_DIR = 'C:\\Users\\delah\\Documents\\Programming\\workspace\\AAForB_Assignment4\\data'
ANALYSIS_DIR = '\\gephi'
os.chdir(DATA_DIR + ANALYSIS_DIR)

nodes_post.to_csv('post_nodes.csv')
nodes_user.to_csv('user_nodes.csv')
nodes_subreddit.to_csv('subreddit_nodes.csv')

rel_user_to_post.to_csv('user_to_post_relationships.csv')
rel_post_to_subreddit.to_csv('post_to_subreddit_relationships.csv')



In [3]:
subreddits

NameError: name 'subreddits' is not defined

# NEO4J QUERIES
## Coronavirus trends

**askreddit**
```
MATCH (p:Post)-[e1]-(s:Subreddit{display_name:'askreddit'})
WHERE p.title contains 'coronavirus' OR p.title contains 'covid'OR p.title contains 'corona virus'
RETURN p.title, s.display_name, p.created_utc_str, p.score
LIMIT 100
```

**science**
```
MATCH (p:Post)-[e1]-(s:Subreddit{display_name:'science'})
WHERE p.title contains 'coronavirus' OR p.title contains 'covid'OR p.title contains 'corona virus'
RETURN p.title, s.display_name, p.created_utc_str, p.score
LIMIT 100
```

**worldnews**
```
MATCH (p:Post)-[e1]-(s:Subreddit{display_name:'worldnews'})
WHERE p.title contains 'coronavirus' OR p.title contains 'covid'OR p.title contains 'corona virus'
RETURN p.title, s.display_name, p.created_utc_str, p.score
LIMIT 100
```

**videos**
```
MATCH (p:Post)-[e1]-(s:Subreddit{display_name:'videos'})
WHERE p.title contains 'coronavirus' OR p.title contains 'covid'OR p.title contains 'corona virus'
RETURN p.title, s.display_name, p.created_utc_str, p.score
LIMIT 100
```
**todayilearned**
```
MATCH (p:Post)-[e1]-(s:Subreddit{display_name:'todayilearned'})
WHERE p.title contains 'coronavirus' OR p.title contains 'covid'OR p.title contains 'corona virus'
RETURN p.title, s.display_name, p.created_utc_str, p.score
LIMIT 100
```
**news**
```
MATCH (p:Post)-[e1]-(s:Subreddit{display_name:'news'})
WHERE p.title contains 'coronavirus' OR p.title contains 'covid'OR p.title contains 'corona virus'
RETURN p.title, s.display_name, p.created_utc_str, p.score
LIMIT 100
```
**iama**
```
MATCH (p:Post)-[e1]-(s:Subreddit{display_name:'iama'})
WHERE p.title contains 'coronavirus' OR p.title contains 'covid'OR p.title contains 'corona virus'
RETURN p.title, s.display_name, p.created_utc_str, p.score
LIMIT 100
```
**askscience**
```
MATCH (p:Post)-[e1]-(s:Subreddit{display_name:'askscience'})
WHERE p.title contains 'coronavirus' OR p.title contains 'covid'OR p.title contains 'corona virus'
RETURN p.title, s.display_name, p.created_utc_str, p.score
LIMIT 100
```
**explainlikeimfive**
```
MATCH (p:Post)-[e1]-(s:Subreddit{display_name:'explainlikeimfive'})
WHERE p.title contains 'coronavirus' OR p.title contains 'covid'OR p.title contains 'corona virus'
RETURN p.title, s.display_name, p.created_utc_str, p.score
LIMIT 100
```
**lifeprotips**
```
MATCH (p:Post)-[e1]-(s:Subreddit{display_name:'lifeprotips'})
WHERE p.title contains 'coronavirus' OR p.title contains 'covid'OR p.title contains 'corona virus'
RETURN p.title, s.display_name, p.created_utc_str, p.score
LIMIT 100
```
**nottheonion**
```
MATCH (p:Post)-[e1]-(s:Subreddit{display_name:'nottheonion'})
WHERE p.title contains 'coronavirus' OR p.title contains 'covid'OR p.title contains 'corona virus'
RETURN p.title, s.display_name, p.created_utc_str, p.score
LIMIT 100
```
**upliftingnews**
```
MATCH (p:Post)-[e1]-(s:Subreddit{display_name:'upliftingnews'})
WHERE p.title contains 'coronavirus' OR p.title contains 'covid'OR p.title contains 'corona virus'
RETURN p.title, s.display_name, p.created_utc_str, p.score
LIMIT 100
```
**dataisbeautiful**
```
MATCH (p:Post)-[e1]-(s:Subreddit{display_name:'dataisbeautiful'})
WHERE p.title contains 'coronavirus' OR p.title contains 'covid'OR p.title contains 'corona virus'
RETURN p.title, s.display_name, p.created_utc_str, p.score
LIMIT 100
```
**technology**
```
MATCH (p:Post)-[e1]-(s:Subreddit{display_name:'technology'})
WHERE p.title contains 'coronavirus' OR p.title contains 'covid'OR p.title contains 'corona virus'
RETURN p.title, s.display_name, p.created_utc_str, p.score
LIMIT 100
```
**politics**
```
MATCH (p:Post)-[e1]-(s:Subreddit{display_name:'politics'})
WHERE p.title contains 'coronavirus' OR p.title contains 'covid'OR p.title contains 'corona virus'
RETURN p.title, s.display_name, p.created_utc_str, p.score
LIMIT 100
```
**europe**
```
MATCH (p:Post)-[e1]-(s:Subreddit{display_name:'europe'})
WHERE p.title contains 'coronavirus' OR p.title contains 'covid'OR p.title contains 'corona virus'
RETURN p.title, s.display_name, p.created_utc_str, p.score
LIMIT 100
```
**coronavirus**
```
MATCH (p:Post)-[e1]-(s:Subreddit{display_name:'coronavirus'})
WHERE p.title contains 'coronavirus' OR p.title contains 'covid'OR p.title contains 'corona virus'
RETURN p.title, s.display_name, p.created_utc_str, p.score
LIMIT 100
```
**covid19**
```
MATCH (p:Post)-[e1]-(s:Subreddit{display_name:'covid19'})
WHERE p.title contains 'coronavirus' OR p.title contains 'covid'OR p.title contains 'corona virus'
RETURN p.title, s.display_name, p.created_utc_str, p.score
LIMIT 100
```
**china_flu**
```
MATCH (p:Post)-[e1]-(s:Subreddit{display_name:'china_flu'})
WHERE p.title contains 'coronavirus' OR p.title contains 'covid'OR p.title contains 'corona virus'
RETURN p.title, s.display_name, p.created_utc_str, p.score
LIMIT 100
```
